In [1]:
import base64
import requests

CLIENT_ID = 'Aa8RibFG1w0qagI7GyjtU10ahPzicLogUGPZFjpaOs_flMZsc1mCkyPVHZsL6G9V41sgPnMTBj-6sfpz'
SECRET = 'EKKGIa7on9MuqluidHC-Bi8orAVe2GnJyS1s-QVmrT6TlLeZ7rWPa-abKu_tjl5Nam_ISfB3QFWWpkyD'

PAYPAL_API = 'https://api-m.paypal.com'


def get_access_token():
    auth = base64.b64encode(f"{CLIENT_ID}:{SECRET}".encode()).decode()
    headers = {
        'Authorization': f'Basic {auth}',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    data = {'grant_type': 'client_credentials'}

    response = requests.post(f"{PAYPAL_API}/v1/oauth2/token", headers=headers, data=data)
    response.raise_for_status()
    return response.json()['access_token']


def create_product(access_token):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }
    product_data = {
        "name": "VIP Subscription",
        "description": "Subscription to VIP status",
        "type": "SERVICE",
        "category": "SOFTWARE"
    }
    response = requests.post(f"{PAYPAL_API}/v1/catalogs/products", headers=headers, json=product_data)
    response.raise_for_status()
    return response.json()


def create_plan(access_token, product_id):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }
    plan_data = {
        "product_id": product_id,
        "name": "Monthly VIP Plan",
        "billing_cycles": [
            {
                "frequency": {
                    "interval_unit": "MONTH",
                    "interval_count": 1
                },
                "tenure_type": "REGULAR",
                "sequence": 1,
                "total_cycles": 0,
                "pricing_scheme": {
                    "fixed_price": {
                        "value": "10",
                        "currency_code": "USD"
                    }
                }
            }
        ],
        "payment_preferences": {
            "auto_bill_outstanding": True,
            "setup_fee_failure_action": "CONTINUE",
            "payment_failure_threshold": 3
        },
        "taxes": {
            "percentage": "0",
            "inclusive": False
        }
    }
    response = requests.post(f"{PAYPAL_API}/v1/billing/plans", headers=headers, json=plan_data)
    response.raise_for_status()
    return response.json()


def main():
    print("Getting access token...")
    token = get_access_token()
    print("Access Token:", token)

    print("Creating product...")
    product = create_product(token)
    print("Product created:", product)

    print("Creating plan...")
    plan = create_plan(token, product['id'])
    print("Plan created:", plan)


if __name__ == "__main__":
    main()


Getting access token...
Access Token: A21AAPDAcUrvEg461r1k0cdz7Kp7l-loVPYAeHak0adqPPHzNBapM52f1RfHr7ca8GTYcorjT1pE4eDc779OYD0wGvj_oRphw
Creating product...
Product created: {'id': 'PROD-55V13643B8312572V', 'name': 'VIP Subscription', 'description': 'Subscription to VIP status', 'create_time': '2025-07-16T20:53:40Z', 'links': [{'href': 'https://api.paypal.com/v1/catalogs/products/PROD-55V13643B8312572V', 'rel': 'self', 'method': 'GET'}, {'href': 'https://api.paypal.com/v1/catalogs/products/PROD-55V13643B8312572V', 'rel': 'edit', 'method': 'PATCH'}]}
Creating plan...
Plan created: {'id': 'P-9KF04931F8882161GNB4BCVA', 'product_id': 'PROD-55V13643B8312572V', 'name': 'Monthly VIP Plan', 'status': 'ACTIVE', 'usage_type': 'LICENSED', 'create_time': '2025-07-16T20:53:40Z', 'links': [{'href': 'https://api.paypal.com/v1/billing/plans/P-9KF04931F8882161GNB4BCVA', 'rel': 'self', 'method': 'GET', 'encType': 'application/json'}, {'href': 'https://api.paypal.com/v1/billing/plans/P-9KF04931F8882161GNB